## 網頁爬取CoinDesk文章

In [17]:
# 載入需要的套件
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
import random


## 爬取文章內連結

In [22]:
#瀏覽器設定，無痕、Fake Agent
opts = Options()
opts.add_argument("--incognito")
ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
opts.add_argument("user-agent={}".format(ua))


# CoinURL:爬取到的網址
CoinURL=[]

# CoinType:爬取到文章類型
CoinType=[]

#設定使用瀏覽器、最大化
driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)
driver.get("https://www.coindesk.com/search?s=bitcoin&sort=1")
driver.maximize_window()


for i in range(1000): 
    driver.implicitly_wait(6) 
    soup = BeautifulSoup(driver.page_source, 'html5lib')   
    allnode_of_a = soup.find_all("a",class_='Box-sc-1hpkeeg-0 hBnhmi')
    result = [_.get("href") for _ in allnode_of_a]
    # Context_type=[_.split('/')[1] for _ in result]
    CoinURL=CoinURL+result
    # CoinType=CoinType+Context_type


    driver.execute_script("window.scrollBy(0, 2500);")

    #使用按鍵到下一頁
    if i ==1:Next=driver.find_element(By.XPATH,'//*[@id="queryly_advanced_container"]/div[5]/div[2]/button[5]/h6')
    else:Next=driver.find_element(By.XPATH,'//*[@id="queryly_advanced_container"]/div[5]/div[2]/button[4]/h6')
    time.sleep(random.randrange(5))
    driver.execute_script("arguments[0].click();", Next)

    if (i%100==0) & (i>0) :
        lo={'CoinURL':CoinURL}
        df = pd.DataFrame(lo)
        df.to_csv('Data_URL/CoinURL-'+str(i)+'.csv')
        print(str(i)+' times done')

lo={'CoinURL':CoinURL}
df = pd.DataFrame(lo)
df.to_csv('Data_URL/CoinURL.csv')

# 關閉瀏覽器視窗
driver.close() 


C:\Users\nptu\AppData\Local\Temp\ipykernel_11184\2466462473.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)
C:\Users\nptu\AppData\Local\Temp\ipykernel_11184\2466462473.py:15: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)


100 times done
200 times done
300 times done
400 times done
500 times done
600 times done
700 times done
800 times done
900 times done


## 在網址取得後，爬取文章內容

In [14]:
from selenium.webdriver.chrome.options import Options

opts = Options()
opts.add_argument("--incognito")  # 使用無痕模式。用 selenium開瀏覽器已經很乾淨了，但疑心病重的可以用一下
ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
opts.add_argument("user-agent={}".format(ua))  



df = pd.read_csv('../DataSet/CoinURL_final.csv')
driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)
CoinText=[]
dataa=len(df['CoinURL'])
for url in range(16402,16436):
    try:
        driver.get("https://www.coindesk.com"+df['CoinURL'][url]) # 更改網址以前往不同網頁
        driver.implicitly_wait(6)
        soup = BeautifulSoup(driver.page_source, 'html5lib')
        allnode_of_a = soup.find_all("div",class_='common-textstyles__StyledWrapper-sc-18pd49k-0 eSbCkN')
        result=["".join([_.getText() for _ in allnode_of_a])]
        CoinText+=result
        print(url)        
    except:
        driver.refresh()
        driver.implicitly_wait(15)
        soup = BeautifulSoup(driver.page_source, 'html5lib')
        allnode_of_a = soup.find_all("div",class_='common-textstyles__StyledWrapper-sc-18pd49k-0 eSbCkN')
        result=["".join([_.getText() for _ in allnode_of_a])]
        CoinText+=result
        print(url)
    if ((url%100)==0) &(url>0):
        driver.close() 
        lo={'CoinText':CoinText,'CoinDate':df['CoinDate'][16402:url+1]}
        df_N = pd.DataFrame(lo)
        df_N.to_csv('CoinText_'+str(url)+'.csv')
        df_N.to_csv('CoinText_final.csv')
        driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)

lo={'CoinText':CoinText,'CoinDate':df['CoinDate'][16402:url+1]}
df_N = pd.DataFrame(lo)
df_N.to_csv('CoinText_final.csv')

driver.close() # 關閉瀏覽器視窗

C:\Users\nptu\AppData\Local\Temp\ipykernel_16084\3838285511.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)
C:\Users\nptu\AppData\Local\Temp\ipykernel_16084\3838285511.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)


16402
16403
16404
16405
16406
16407
16408
16409
16410
16411
16412
16413
16414
16415
16416
16417
16418
16419
16420
16421
16422
16423
16424
16425
16426
16427
16428
16429
16430
16431
16432
16433
16434
16435


In [12]:
df = pd.read_csv('../DataSet/CoinURL_final.csv')
df['CoinURL'][16435]

'/markets/2014/10/01/digitaltangible-expands-crypto-20-gold-trading-platform-to-asia/'